In [1]:
import numpy as np
import seaborn as sns
import pandas as pd

In [2]:
pd.set_option("display.max_columns", None)

covid_url = "https://dqlab.id/data/covid19_worldwide_2020.json"
df_covid_worldwide = pd.read_json(covid_url)

In [4]:
df_covid_worldwide.head()

,date,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths,geo_id
0,2020-03-20,1,0,1,0,TD
1,2020-03-21,0,0,1,0,TD
2,2020-03-22,1,0,2,0,TD
3,2020-03-23,0,0,2,0,TD
4,2020-03-24,0,0,2,0,TD


In [6]:
df_covid_worldwide.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61900 entries, 0 to 61899
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   61900 non-null  datetime64[ns]
 1   daily_confirmed_cases  61900 non-null  int64         
 2   daily_deaths           61900 non-null  int64         
 3   confirmed_cases        61900 non-null  int64         
 4   deaths                 61900 non-null  int64         
 5   geo_id                 61625 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 3.3+ MB


In [5]:
df_covid_worldwide.set_index("date").sort_index()

,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths,geo_id
date,,,,,
2019-12-31,0,0,0,0,IN
2019-12-31,0,0,0,0,IR
2019-12-31,0,0,0,0,PK
2019-12-31,0,0,0,0,AM
2019-12-31,27,0,27,0,CN
...,...,...,...,...,...
2020-12-14,528,1,9708,92,UY
2020-12-14,26,0,69,0,GD
2020-12-14,0,0,11101,166,MT


In [8]:
print("Jumlah missing value tiap kolom:")
print(df_covid_worldwide.isna().sum())

df_covid_worldwide.dropna(inplace=True)

Jumlah missing value tiap kolom:
date                       0
daily_confirmed_cases      0
daily_deaths               0
confirmed_cases            0
deaths                     0
geo_id                   275
dtype: int64


In [9]:
# setelah dihapus
df_covid_worldwide.isna().sum()

date                     0
daily_confirmed_cases    0
daily_deaths             0
confirmed_cases          0
deaths                   0
geo_id                   0
dtype: int64

In [10]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

countries_url = "https://dqlab.id/data/country_details.json"
df_countries = pd.read_json(countries_url)
print(df_countries.head())

  geo_id country_name
0     TD         Chad
1     CU         Cuba
2     FJ         Fiji
3     GU         Guam
4     IR         Iran


Selanjutnya adalah mapping data covid19 dan data country. Gunakan fungsi .merge pada pandas untuk menggabungkan df_covid_worldwide dan df_countries. 

Untuk merge, gunakan kolom geo_id, (ingat ya df_covid_worldwide memiliki index pada kolom date untuk itu perlu .reset_index()). Setelah merge dapat diset kembali index ke kolom date.

In [11]:
df_covid_denormalized = pd.merge(df_covid_worldwide.reset_index(), df_countries, on="geo_id").set_index("date")
print(df_covid_denormalized.head())

            index  daily_confirmed_cases  daily_deaths  confirmed_cases  \
date                                                                      
2020-03-20      0                      1             0                1   
2020-03-21      1                      0             0                1   
2020-03-22      2                      1             0                2   
2020-03-23      3                      0             0                2   
2020-03-24      4                      0             0                2   

            deaths geo_id country_name  
date                                    
2020-03-20       0     TD         Chad  
2020-03-21       0     TD         Chad  
2020-03-22       0     TD         Chad  
2020-03-23       0     TD         Chad  
2020-03-24       0     TD         Chad  


In [12]:
# Tambahkan satu kolom tambahan yaitu fatality_ratio, yang merupakan pembagian antara kolom deaths dan confirmed_cases.
df_covid_denormalized["deaths"]/df_covid_denormalized["confirmed_cases"]
df_covid_denormalized.head()

,index,daily_confirmed_cases,daily_deaths,confirmed_cases,deaths,geo_id,country_name
date,,,,,,,
2020-03-20,0,1,0,1,0,TD,Chad
2020-03-21,1,0,0,1,0,TD,Chad
2020-03-22,2,1,0,2,0,TD,Chad
2020-03-23,3,0,0,2,0,TD,Chad
2020-03-24,4,0,0,2,0,TD,Chad


In [13]:
df_covid_denormalized_august = df_covid_denormalized.loc["2020-08"].groupby("country_name").sum()

df_covid_denormalized_august["fatality_ratio"] = df_covid_denormalized_august["deaths"]/df_covid_denormalized_august["confirmed_cases"]

df_top_20_fatality_rate_on_august = df_covid_denormalized_august.sort_values(by="fatality_ratio", ascending=False).head(20)
print(df_top_20_fatality_rate_on_august["fatality_ratio"])

country_name
Yemen                     0.286473
Italy                     0.138619
France                    0.137979
United_Kingdom            0.130228
Belgium                   0.125079
Hungary                   0.122820
Netherlands               0.099309
Mexico                    0.099092
Jersey                    0.088677
Spain                     0.079746
Chad                      0.078870
Montserrat                0.076923
Canada                    0.073963
Sweden                    0.071475
Isle_of_Man               0.071429
British_Virgin_Islands    0.066239
Ireland                   0.065030
Sudan                     0.064250
Liberia                   0.063944
Ecuador                   0.061031
Name: fatality_ratio, dtype: float64
